In [2]:
import pandas as pd

In [3]:
train=pd.read_csv("train_ctrUa4K.csv")
train=train.drop("Loan_ID",1)
test=pd.read_csv("test_lAUu6dG.csv")

In [4]:
train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
X=train.drop("Loan_Status",1)
y=train["Loan_Status"]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
numeric_transformer=Pipeline(steps=[("imputer",SimpleImputer(strategy="median")),("scalar",StandardScaler())])
categorical_transformer=Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent")),("encoding",OneHotEncoder(handle_unknown="ignore"))])

In [24]:
from sklearn.compose import ColumnTransformer
numeric_list=X.select_dtypes(include=["int64","float64"]).columns
numeric_list
categorical_list=X.select_dtypes(include=["object"]).columns
categorical_list
preprocessor=ColumnTransformer(transformers=[("num",numeric_transformer,numeric_list),("cat",categorical_transformer,categorical_list)])

In [25]:
from sklearn.ensemble import RandomForestClassifier
rf=Pipeline(steps=[("preprocess",preprocessor),("classifier",RandomForestClassifier())])

In [26]:
rf.fit(X_train,y_train)

c:\users\jagpreet\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                               

In [27]:
predict=rf.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(predict,y_test)

0.7642276422764228

In [29]:
param_grid = { 
    'classifier__n_estimators': [200, 500],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth' : [4,5,6,7,8],
    'classifier__criterion' :['gini', 'entropy'],
     'classifier__min_samples_leaf' :[1,2,3,4,5,6,7,8,9,10]     }

In [30]:
from sklearn.model_selection import GridSearchCV
CV = GridSearchCV(rf, param_grid, n_jobs=-1,verbose=3)
                  
CV.fit(X_train, y_train)  
print(CV.best_params_)    
print(CV.best_score_)

c:\users\jagpreet\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 600 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:  5.9min finished


{'classifier__criterion': 'gini', 'classifier__max_depth': 4, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 3, 'classifier__n_estimators': 200}
0.8004073319755601
